In [150]:
import math
from itertools import product
from collections import defaultdict
import matplotlib as plt
import shapely
from shapely.geometry import LineString, MultiLineString

In [30]:
test = [2.8533979930661495, 0.8350593062280169, 1.8350735010944388]
lines = [[[(90.92466560343746, 359.8743795603514), (50.64634319592733, 342.689586349763), (141.4154383126297, 0.0), (738.6390963118174, 170.50443049520254)], [(528.746321437764, 592.3512355554849), (250.68498866126174, 548.8781394008547)]], [[(738.6390963118174, 170.50443049520254), (1041.5171545767225, 487.5379093941301), (733.3747470285161, 795.9299552580342), (528.746321437764, 592.3512355554849)]], [[(250.68498866126174, 548.8781394008547), (148.7872773281997, 925.0354834403843), (84.8542414731346, 907.106489722617)], [(84.8542414731346, 907.106489722617), (75.55369137861999, 940.1296991426498), (9.845574646664318, 921.7006750889122), (72.29789360892028, 700.0236357413232)], [(72.29789360892028, 700.0236357413232), (0.0, 679.7554941019043), (90.92466560343746, 359.8743795603514)]]]

In [165]:
class BaseSection:
    def __init__(self, angle, leng, widt, road):
        self.angle = angle
        self.an_normalized = self.angle_normalized()
        self.length, self.width, self.road = leng, widt, road
        self.width_dep = self.width_dependency()
        self.length_r, self.width_r = self.possible_combinations()

    def angle_normalized(self):
        if self.angle > math.pi / 4:
            if self.angle < math.pi / 2:
                ang = (math.pi / 2) - self.angle
            elif 3 * math.pi / 4 > self.angle > math.pi / 2:
                ang = self.angle - (math.pi / 2)
            else:
                ang = math.pi - self.angle
        else:
            ang = self.angle
        return ang

    def width_dependency(self):
        width = {}
        for key, value in self.width.items():
            new_w = value[0] + (((value[1] - value[0]) * self.an_normalized) / 0.872665)  # 50 degrees
            width[key] = [round(new_w)]
        return width

    def possible_combinations(self):
        new_length = defaultdict(list)
        new_width = defaultdict(list)
        for key in self.road.keys():
            l = list(product(self.road[key], self.length[key], repeat = 1))
            new_length[key] = [sum(i) for i in l]
            w = list(product(self.road[key], self.width[key], repeat = 1))
            new_width[key] = [sum(i) for i in w]

        return new_length, new_width


In [166]:
input_types = {'9':[54,74], '12':[62,89]}
widths = {'9':[72, 90, 108, 126, 144], '12':[65, 81, 98, 114, 130]}
roads = {'9':[68.8, 35.6], '12': [76, 40.6]}

tes = BaseSection(test[2], input_types, widths, roads)
a, b = tes.length_r, tes.width_r
print(a, b)

defaultdict(<class 'list'>, {'9': [122.8, 142.8, 89.6, 109.6], '12': [138, 165, 102.6, 129.6]}) defaultdict(<class 'list'>, {'9': [140.8, 158.8, 176.8, 194.8, 212.8, 107.6, 125.6, 143.6, 161.6, 179.6], '12': [141, 157, 174, 190, 206, 105.6, 121.6, 138.6, 154.6, 170.6]})


In [100]:
class PolyClusters:
    def __init__(self, coordinates, angle, **kwargs):
        self.coordinates = coordinates
        self.angle = angle
        self.line = MultiLineString([LineString(c) for c in coordinates])
        self.boundrec = list(self.line.minimum_rotated_rectangle.exterior.coords)
        self.length = self.seg_length()
        self.basesec = BaseSection(angle, **kwargs)
        self.sum = self.grid_options()

    # dolzhno bit opisano ranee, pomoika
    def seg_length(self):
        l = LineString([self.boundrec[1], self.boundrec[2]]).length
        return round(l)

    def grid_options(self):

        def combination_sum():
            ans = []
            temp = []
            arr = sorted(list([i[0] for i in self.basesec.width_dep.values()]))
            #arr_vals = list(self.basesec.width_dep.values())
            #arr_keys = list(self.basesec.width_dep.keys())
            ln = self.length
            find_numbers(ans, arr, temp, ln, 0)
            return ans

        def find_numbers(ans, arr, temp, ln, index):
            if arr[0] / 3 > ln >= 0:
                ans.append([list(temp), ln])
            for i in range(index, len(arr)):
                if (ln - arr[i]) >= 0:
                    #temp.append(int(arr_keys[arr_vals.index(arr[i])]) * 4.6)
                    temp.append(arr[i])
                    find_numbers(ans, arr, temp, ln - arr[i], i)
                    #temp.remove(int(arr_keys[arr_vals.index(arr[i])]) * 4.6)
                    temp.remove(arr[i])

        return combination_sum()


вопрос как обойтись со стандартными параметрами типо ширин проездов и т д?

In [103]:
input_types = {'9':[54,74], '12':[62,89]}
widths = {'9':[72, 90, 108, 126, 144], '12':[65, 81, 98, 114, 130]}
roads = {'9':[68.8, 35.6], '12': [76, 40.6]}


ii = PolyClusters(lines[2], test[2], **input_types)
leng = ii.length
iii = BaseSection(test[2], **input_types)
nt = iii.width_dep

dorg = ii.sum
dorg

[[[60, 60, 70], 18], [[60, 70, 70], 8]]

[[128.8, 95.6], [146, 110.6]] [[140.8, 158.8, 176.8, 194.8, 212.8, 107.6, 125.6, 143.6, 161.6, 179.6], [141, 157, 174, 190, 206, 105.6, 121.6, 138.6, 154.6, 170.6]]
